# Eodag advanced features
_____________

We present `eodag`'s advanced features for **searching and downloading EO products**.

To be able to follow this tutorial, you will need to install the additional Python packages [ipyleaflet](https://ipyleaflet.readthedocs.io/en/latest/installation.html) and [ipywidgets](https://ipywidgets.readthedocs.io/en/latest/user_install.html).

## Configuration

Let's start by setting your personal credentials to access [sobloo service](https://sobloo.eu/) by filling your API key below:

In [1]:
import os
from pprint import pprint
os.environ["EODAG__SOBLOO__AUTH__CREDENTIALS__APIKEY"] = "PLEASE_CHANGE_ME"

The name of the environment variables understood by `eodag` must follow the pattern `EODAG__KEY1__KEY2__[...]__KEYN` (note the double underscore between the keys).

See for instance the following configuration extracted from a minimal YAML file:

```yaml
sobloo:
    download:
        extract: True
        outputs_prefix: <absolute_path_to_download_folder>

```

It sets the `extract` and `outputs_prefix` (download folder) parameters, which could also be set with the environment variables `EODAG__SOBLOO__DOWNLOAD__EXTRACT` and `EODAG__SOBLOO__DOWNLOAD__OUTPUTS_PREFIX` given the established pattern. Every configuration parameter can be changed this way. Check the YAML file located at `~/.config/eodag/eodag.yml` to see all the parameters you can set.

We create a workspace directory where all our files will live:

In [2]:
workspace = 'eodag_workspace_advanced'
if not os.path.isdir(workspace):
    os.mkdir(workspace)

Let's check that the Python packages required to run this notebook are available:

In [3]:
import ipyleaflet as ipyl
import ipywidgets as ipyw

We configure `eodag` with environment variables to download *sobloo* products to our workspace folder but not to extract them once they are downloaded. Note that in this tutorial we do not use a YAML configuration as demonstrated in the basic example. The complete configuration is done by setting the right environment variables.

In [4]:
os.environ["EODAG__SOBLOO__DOWNLOAD__OUTPUTS_PREFIX"] = os.path.abspath(workspace)
os.environ["EODAG__SOBLOO__DOWNLOAD__EXTRACT"] =  "false"

We are now ready to initialize the session by creating an `EODataAccessGateway` instance which is going to take into account what we have just configured.

In [5]:
from eodag.api.core import EODataAccessGateway
from eodag.utils.logging import setup_logging

# To have some basic feedback on what eodag is doing, we configure logging to output minimum information
setup_logging(verbose=1)
dag = EODataAccessGateway()

2021-01-26 16:54:21,617-15s eodag.config                     [INFO    ] Loading user configuration from: /home/sylvain/.config/eodag/eodag.yml
2021-01-26 16:54:21,994-15s eodag.core                       [INFO    ] Locations configuration loaded from /home/sylvain/.config/eodag/locations.yml


`eodag` includes a number of different providers (*PEPS*, *USGS*, *sobloo*, etc.). Each provider has a priority expressed as an number, the lower this number the lower the priority of the provider. The one with the higher priority is used to search and download products.

*PEPS* is defined as the default provider. We can easily verify that:

In [6]:
print(dag.get_preferred_provider())

('peps', 1)


Since we want to search for products from *sobloo*, we have to set it as the the preferred provider:

In [7]:
dag.set_preferred_provider("sobloo")
print(dag.get_preferred_provider())

('sobloo', 2)


The priority of the providers can also be defined in a YAML configuration file and with environment variables.

## Search

Now let's search for Sentinel-2 L1C products in the South of France acquired in the last 30 days.

We will add two complementary search parameters. First we would like to filter at 10% of maximum cloud cover. In order to do so we can set the parameter `cloudCover` which is an OGC standard parameter. Second we would like to search for products that are immediately available for download. `eodag` defines this kind of product as *ONLINE* (see the Introduction page for more details). The `search` method accepts any number of keyword arguments. This feature is handy to define this setting since its syntax is specific to *sobloo*:

In [8]:
from datetime import date, timedelta

product_type = 'S2_MSI_L1C'
extent = {
    'lonmin': -1.999512,
    'lonmax': 4.570313,
    'latmin': 42.763146,
    'latmax': 46.754917
}

products, estimated_total_nbr_of_results = dag.search(
    productType=product_type,
    start=(date.today() - timedelta(days=30)).isoformat(),
    end=date.today().isoformat(),
    geom=extent,
    cloudCover=10,
    **{"state.services.download": "internal"}
)
pprint(products)

2021-01-26 16:54:22,216-15s eodag.core                       [INFO    ] Searching product type 'S2_MSI_L1C' on provider: sobloo
2021-01-26 16:54:22,218-15s eodag.plugins.search.qssearch    [INFO    ] Sending count request: https://sobloo.eu/api/v1/services/search?f=contentDescription.cloudCoverPercentage:lte:10&f=state%2Eservices%2Edownload:eq:internal&f=acquisition.beginViewingDate:gte:1609023600000&f=acquisition.endViewingDate:lte:1611615600000&f=identification.type:eq:S2MSI1C&gintersect=POLYGON ((-1.9995 42.7631, -1.9995 46.7549, 4.5703 46.7549, 4.5703 42.7631, -1.9995 42.7631))&size=1&from=0
2021-01-26 16:54:22,932-15s eodag.plugins.search.qssearch    [INFO    ] Sending search request: https://sobloo.eu/api/v1/services/search?f=contentDescription.cloudCoverPercentage:lte:10&f=state%2Eservices%2Edownload:eq:internal&f=acquisition.beginViewingDate:gte:1609023600000&f=acquisition.endViewingDate:lte:1611615600000&f=identification.type:eq:S2MSI1C&gintersect=POLYGON ((-1.9995 42.7631, -1

[EOProduct(id=S2B_MSIL1C_20210103T110349_N0209_R094_T30TYP_20210103T121021, provider=sobloo),
 EOProduct(id=S2A_MSIL1C_20210108T110431_N0209_R094_T31TCK_20210108T131314, provider=sobloo),
 EOProduct(id=S2A_MSIL1C_20210108T110431_N0209_R094_T30TXS_20210108T131314, provider=sobloo),
 EOProduct(id=S2B_MSIL1C_20210114T103309_N0209_R108_T31TFH_20210114T114331, provider=sobloo),
 EOProduct(id=S2B_MSIL1C_20210114T103309_N0209_R108_T31TEH_20210114T114331, provider=sobloo),
 EOProduct(id=S2A_MSIL1C_20210118T110401_N0209_R094_T30TYP_20210118T131217, provider=sobloo),
 EOProduct(id=S2A_MSIL1C_20210118T110401_N0209_R094_T30TYQ_20210118T131217, provider=sobloo),
 EOProduct(id=S2A_MSIL1C_20210118T110401_N0209_R094_T30TXR_20210118T131217, provider=sobloo),
 EOProduct(id=S2B_MSIL1C_20210117T104259_N0209_R008_T31TFJ_20210117T115034, provider=sobloo),
 EOProduct(id=S2B_MSIL1C_20210110T105329_N0209_R051_T30TYQ_20210110T130049, provider=sobloo),
 EOProduct(id=S2B_MSIL1C_20210110T105329_N0209_R051_T30TYS_2

We can check that the products we have got have a cloud cover below 10% and that they are all immediately available for download.

In [9]:
all(p.properties["cloudCover"] < 10 for p in products)

True

In [10]:
all(p.properties["storageStatus"] == "ONLINE" for p in products)

True

As already demonstrated in the basic example we use `ipyleaflet` to plot the returned results (in blue). This time we also map our search extent (in green): 

In [11]:
import ipyleaflet as ipyl

m = ipyl.Map(center=[43.6, 1.5], zoom=5)
search_layer = ipyl.Rectangle(
    bounds=((extent["latmin"], extent["lonmin"]), (extent["latmax"], extent["lonmax"])),
    color="green"
)
products_layer = ipyl.GeoJSON(data=products.as_geojson_object(), hover_style={'fillColor': 'yellow'})
m.add_layer(search_layer)
m.add_layer(products_layer)
m

Map(center=[43.6, 1.5], controls=(ZoomControl(options=['position', 'zoom_in_text', 'zoom_in_title', 'zoom_out_…

`eodag` includes several *crunchers* that allow you to filter products:
* `FilterDate`: by start date (optional) and end date (optional)
* `FilterLatestByName`: get only the latest product based on the name of the product
* `FilterLatestIntersect`: filter latest products (the ones with a the highest start date) that intersect search extent
* `FilterOverlap`: retain only those that are overlapping with a search extent
* `FilterProperty`: retain only those whose property match a given criteria

In this example we demonstrate how to use the cruncher `FilterOverlap` to get the products that are strictly within the search bounding box. For more information about the crunchers checkout the API documentation.

In [12]:
from eodag.plugins.crunch.filter_overlap import FilterOverlap

filtered_products = products.crunch(FilterOverlap({"minimum_overlap": 100}), geometry=extent)

2021-01-26 16:54:23,905-15s eodag.plugins.crunch.filter_overlap [INFO    ] Finished filtering products. 8 resulting products


In [13]:
pprint(filtered_products)

[EOProduct(id=S2B_MSIL1C_20210103T110349_N0209_R094_T30TYP_20210103T121021, provider=sobloo),
 EOProduct(id=S2A_MSIL1C_20210108T110431_N0209_R094_T31TCK_20210108T131314, provider=sobloo),
 EOProduct(id=S2A_MSIL1C_20210118T110401_N0209_R094_T30TYP_20210118T131217, provider=sobloo),
 EOProduct(id=S2A_MSIL1C_20210118T110401_N0209_R094_T30TYQ_20210118T131217, provider=sobloo),
 EOProduct(id=S2A_MSIL1C_20210118T110401_N0209_R094_T30TXR_20210118T131217, provider=sobloo),
 EOProduct(id=S2B_MSIL1C_20210110T105329_N0209_R051_T30TYQ_20210110T130049, provider=sobloo),
 EOProduct(id=S2B_MSIL1C_20210110T105329_N0209_R051_T31TEJ_20210110T130049, provider=sobloo),
 EOProduct(id=S2A_MSIL1C_20210108T110431_N0209_R094_T30TXQ_20210108T131314, provider=sobloo)]


We add the filtered products to the **previous map** (in red).

In [14]:
filtered_products_layer = ipyl.GeoJSON(
    data=filtered_products.as_geojson_object(), 
    hover_style={'fillColor': 'yellow'},
    style={"color":"red"}
)
m.add_layer(filtered_products_layer)

We create a new map which displays the product *ID* on hover. It also allows you to select which products you would like to download. To try it out, click on a filtered product (in red) to add it to the download list `chosen_products`. Once a product is selected, its contour line should turn yellow. Note that the more products you select on the map the longer the download time in the end.

In [15]:
from eodag.api.product import EOProduct
from eodag.api.search_result import SearchResult

label = ipyw.Label(layout=ipyw.Layout(width='100%'))

chosen_products = SearchResult([])

def click_handler(feature=None, **kwargs):
    if feature and feature not in chosen_products:
        chosen_products.append(EOProduct.from_geojson(feature))
        # register product downloader
        chosen_products[-1].register_downloader(products[0].downloader, products[0].downloader_auth)
        # update map
        chosen_products_layer = ipyl.GeoJSON(
            data=chosen_products.as_geojson_object(), 
            hover_style={'fillColor': 'yellow'},
            style={"color":"yellow"}
        )
        if chosen_products_layer in m.layers:
            m.remove_layer(chosen_products_layer)
        m.add_layer(chosen_products_layer)
        
filtered_products_layer.on_click(click_handler)

def hover_handler(id=None, **kwargs):
    label.value = id
filtered_products_layer.on_hover(hover_handler)

ipyw.VBox([m, label])

Now that you have selected at least one product the list of `chosen_products` shouldn't be empty anymore.

In [18]:
chosen_products

[EOProduct(id=S2B_MSIL1C_20210110T105329_N0209_R051_T31TEJ_20210110T130049, provider=sobloo), EOProduct(id=S2A_MSIL1C_20210108T110431_N0209_R094_T31TCK_20210108T131314, provider=sobloo)]

## Download

We finally use the `download_all` method to download all the selected products.

In [19]:
from eodag.utils import NotebookProgressCallback

if chosen_products:
    paths = dag.download_all(
        chosen_products, 
        progress_callback=NotebookProgressCallback(), 
    )

2021-01-26 16:55:01,432-15s eodag.core                       [INFO    ] Downloading 2 products


0.00B [00:00, ?B/s]

2021-01-26 16:55:11,781-15s eodag.plugins.download.base      [INFO    ] Extraction not activated. The product is available as is.
2021-01-26 16:56:47,779-15s eodag.plugins.download.base      [INFO    ] Extraction not activated. The product is available as is.


You can now check your workspace folder where you should find the archive files (since we have configured `eodag` not to extract them) of your selected products.

We specified earlier that we wanted *ONLINE* products only. If we hadn't done so some products could have been *OFFLINE*, i.e. not immediately available for download. In this case `edoag` would have first ordered them, and then would have retried to download them every 2 minutes, and kept retrying for 20 minutes. These default times can be changed, for instance, `wait=0.5` and `timeout=5` define them to 30 seconds and 5 minutes respectively.

This concludes the tutorial for the advanced usage of eodag.